In [3]:
import os
import glob
import re
import shutil
import pandas as pd
import copy

In [48]:
df=pd.DataFrame(columns=["PatientID","Pulmonary tumour", "Pulmonary embolism", "Lymphoglandular metastasis", "Pulmonary metastasis", "Covid", "Emphysema", "Bronchiectasia", "Cardiomegalia"])
default_patient={disease:"" for disease in df}
cnt={disease:0 for disease in df}

In [49]:
special_remover = '(\s+)|(\.)|(\-)'
replace_special = ''

In [50]:
neg=r"((?!(nem|nincs)).*)"
regexes= {
    "Pulmonary tumour": fr"tu(pulm|plum){neg}",
    "Pulmonary embolism": fr"emboliapulm{neg}",
    "Lymphoglandular metastasis": fr"metlgl{neg}",
    "Pulmonary metastasis": fr"metpulm{neg}",
    "Covid": fr"covid((19)?)(((tipusu)|(típusu)|(típusú))?)(pneumo((m)?)ni((a)?)){neg}",
    "Emphysema": fr"emphysema{neg}",
    "Bronchiectasia": fr"bronchiectasia{neg}",
    "Cardiomegalia": fr"((cardiomegalia)|(((sziv)|(szív))nagyobb)){neg}"
}

In [51]:
for patient_path in glob.iglob('/mnt/idms/PROJECTS/Lung/Tudo-Ulyssys-Unzipped/*'):
    txt_path=glob.glob(f'{patient_path}/*.txt')[0]
    with open(txt_path, 'r', encoding='latin1', errors='ignore') as file:
        data = file.read().lower()
        data,_ = re.subn(special_remover, replace_special, data)
        patient=copy.deepcopy(default_patient)
        patient["PatientID"]=patient_path
        for disease, regex in regexes.items():
            if re.search(regex, data) is not None:
                patient[disease]="1"
                cnt[disease]+=1
        df=df.append(pd.Series(patient),ignore_index=True)

In [52]:
cnt["PatientID"]="Sum"

In [53]:
df_stat=df.append(pd.Series({k:str(v) for k,v in cnt.items()}), ignore_index=True)

In [54]:
df_stat.to_csv('./Statistics.csv', index=False)

In [55]:
df.to_csv('./Verdicts.csv',index=False)